In [1]:
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Tuple
import re
from collections import defaultdict
from keybert import KeyBERT
from tqdm.notebook import tqdm
import time
import pickle
from datetime import datetime

In [2]:
# Load dataset
ds = load_dataset("SemEvalWorkshop/sem_eval_2010_task_8")

# Take first 10 samples for testing
test_samples = list(ds['train'].select(range(10)))

# Display first sample structure
print("Sample data structure:")
print(test_samples[0])

# Display all 10 sentences and their entities
print("\nSamples with their entities:")
for i, sample in enumerate(test_samples):
    entities = re.findall(r'<e[12]>(.*?)</e[12]>', sample['sentence'])
    print(f"\n{i+1}. Sentence: {sample['sentence']}")
    print(f"   Entities: {entities}")

Sample data structure:
{'sentence': 'The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.', 'relation': 3}

Samples with their entities:

1. Sentence: The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.
   Entities: ['configuration', 'elements']

2. Sentence: The <e1>child</e1> was carefully wrapped and bound into the <e2>cradle</e2> by means of a cord.
   Entities: ['child', 'cradle']

3. Sentence: The <e1>author</e1> of a keygen uses a <e2>disassembler</e2> to look at the raw assembly code.
   Entities: ['author', 'disassembler']

4. Sentence: A misty <e1>ridge</e1> uprises from the <e2>surge</e2>.
   Entities: ['ridge', 'surge']

5. Sentence: The <e1>student</e1> <e2>association</e2> is the voice of the undergraduate student population of the State University of New York at Buffalo.
   Entities: ['student', 'association']

6. Sentence: Th

In [3]:
def clean_sentence(text: str) -> str:
    """Remove XML tags from text"""
    return re.sub(r'</?e[12]>', '', text).strip()

def extract_keyphrases(text: str, model: KeyBERT, top_n: int = 2) -> List[str]:
    """Extract keyphrases using KeyBERT"""
    keyphrases = model.extract_keywords(text, 
                                      keyphrase_ngram_range=(1, 2),
                                      stop_words='english',
                                      top_n=top_n,
                                      use_maxsum=True)
    return [k[0] for k in keyphrases]

# Initialize model
print("Loading KeyBERT model...")
model = KeyBERT()

Loading KeyBERT model...


In [4]:
# Process test samples
for i, sample in enumerate(test_samples):
    # Get original sentence and true entities
    sentence = sample['sentence']
    true_entities = re.findall(r'<e[12]>(.*?)</e[12]>', sentence)
    
    # Clean sentence and extract keyphrases
    clean_text = clean_sentence(sentence)
    extracted_phrases = extract_keyphrases(clean_text, model)
    
    # Print results
    print(f"\nSample {i+1}:")
    print(f"Clean text: {clean_text}")
    print(f"True entities: {true_entities}")
    print(f"Extracted keyphrases: {extracted_phrases}")
    print("-" * 80)


Sample 1:
Clean text: The system as described above has its greatest application in an arrayed configuration of antenna elements.
True entities: ['configuration', 'elements']
Extracted keyphrases: ['greatest', 'antenna elements']
--------------------------------------------------------------------------------

Sample 2:
Clean text: The child was carefully wrapped and bound into the cradle by means of a cord.
True entities: ['child', 'cradle']
Extracted keyphrases: ['means', 'child carefully']
--------------------------------------------------------------------------------

Sample 3:
Clean text: The author of a keygen uses a disassembler to look at the raw assembly code.
True entities: ['author', 'disassembler']
Extracted keyphrases: ['look raw', 'assembly code']
--------------------------------------------------------------------------------

Sample 4:
Clean text: A misty ridge uprises from the surge.
True entities: ['ridge', 'surge']
Extracted keyphrases: ['misty', 'uprises surge']
-

In [5]:
def evaluate_matches(true_entities: List[str], extracted_phrases: List[str], partial_match: bool = True) -> Tuple[float, float, float]:
    """Calculate precision, recall, and F1 score"""
    if partial_match:
        # Count each match only once by tracking which true entities have been matched
        matched_true = set()
        matched_extracted = set()
        
        for i, ext in enumerate(extracted_phrases):
            for j, gold in enumerate(true_entities):
                if (ext.lower() in gold.lower() or gold.lower() in ext.lower()):
                    matched_extracted.add(i)
                    matched_true.add(j)
        
        matches = len(matched_true)  # Count unique matches
    else:
        matches = sum(1 for ext in extracted_phrases 
                     if any(ext.lower() == gold.lower() for gold in true_entities))
    
    precision = matches / len(extracted_phrases) if extracted_phrases else 0
    recall = matches / len(true_entities) if true_entities else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Evaluate all samples
all_metrics = []
for i, sample in enumerate(test_samples):
    # Get entities and extract keyphrases
    true_entities = re.findall(r'<e[12]>(.*?)</e[12]>', sample['sentence'])
    clean_text = clean_sentence(sample['sentence'])
    extracted_phrases = extract_keyphrases(clean_text, model)
    
    # Calculate metrics
    precision, recall, f1 = evaluate_matches(true_entities, extracted_phrases)
    all_metrics.append((precision, recall, f1))
    
    # Print results
    print(f"\nSample {i+1}:")
    print(f"True entities: {true_entities}")
    print(f"Extracted phrases: {extracted_phrases}")
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}")

# Calculate average metrics
avg_precision = np.mean([m[0] for m in all_metrics])
avg_recall = np.mean([m[1] for m in all_metrics])
avg_f1 = np.mean([m[2] for m in all_metrics])

print("\nAverage Metrics:")
print(f"Precision: {avg_precision:.2f}")
print(f"Recall: {avg_recall:.2f}")
print(f"F1 Score: {avg_f1:.2f}")


Sample 1:
True entities: ['configuration', 'elements']
Extracted phrases: ['greatest', 'antenna elements']
Precision: 0.50, Recall: 0.50, F1: 0.50

Sample 2:
True entities: ['child', 'cradle']
Extracted phrases: ['means', 'child carefully']
Precision: 0.50, Recall: 0.50, F1: 0.50

Sample 3:
True entities: ['author', 'disassembler']
Extracted phrases: ['look raw', 'assembly code']
Precision: 0.00, Recall: 0.00, F1: 0.00

Sample 4:
True entities: ['ridge', 'surge']
Extracted phrases: ['misty', 'uprises surge']
Precision: 0.50, Recall: 0.50, F1: 0.50

Sample 5:
True entities: ['student', 'association']
Extracted phrases: ['association voice', 'student population']
Precision: 1.00, Recall: 1.00, F1: 1.00

Sample 6:
True entities: ['complex', 'producer']
Extracted phrases: ['sprawling complex', 'producer silver']
Precision: 1.00, Recall: 1.00, F1: 1.00

Sample 7:
True entities: ['inflammation', 'infection']
Extracted phrases: ['acid production', 'inflammation distal']
Precision: 0.50, Reca

In [6]:
def process_dataset_in_batches(dataset, model, batch_size=200, save_every=1000):
    """
    Process the dataset in batches with progress monitoring and periodic saving
    """
    start_time = time.time()
    all_metrics = []
    all_results = []
    total_samples = len(dataset)
    
    # Create progress bar
    pbar = tqdm(total=total_samples, desc="Processing samples")
    
    for i in range(0, total_samples, batch_size):
        # Get batch
        batch = dataset.select(range(i, min(i + batch_size, total_samples)))
        batch_metrics = []
        batch_results = []
        
        # Process each sample in batch
        for sample in batch:
            true_entities = re.findall(r'<e[12]>(.*?)</e[12]>', sample['sentence'])
            clean_text = clean_sentence(sample['sentence'])
            extracted_phrases = extract_keyphrases(clean_text, model)
            
            # Calculate metrics
            precision, recall, f1 = evaluate_matches(true_entities, extracted_phrases)
            batch_metrics.append((precision, recall, f1))
            
            # Store detailed results
            batch_results.append({
                'sentence': sample['sentence'],
                'true_entities': true_entities,
                'extracted_phrases': extracted_phrases,
                'metrics': {'precision': precision, 'recall': recall, 'f1': f1}
            })
        
        # Update main lists
        all_metrics.extend(batch_metrics)
        all_results.extend(batch_results)
        
        # Update progress bar
        pbar.update(len(batch))
        
        # Save periodically
        if (i + batch_size) % save_every == 0:
            save_results(all_results, all_metrics, i + batch_size)
            
    pbar.close()
    
    # Calculate final averages
    avg_metrics = calculate_average_metrics(all_metrics)
    
    # Print time taken
    time_taken = time.time() - start_time
    print(f"\nTotal time taken: {time_taken:.2f} seconds")
    
    return all_results, avg_metrics

def save_results(results, metrics, samples_processed):
    """Save results to file"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"keyphrase_results_{samples_processed}_{timestamp}.pkl"
    
    with open(filename, 'wb') as f:
        pickle.dump({
            'results': results,
            'metrics': metrics,
            'samples_processed': samples_processed
        }, f)
    print(f"\nSaved results to {filename}")

def calculate_average_metrics(metrics):
    """Calculate average metrics from list of (precision, recall, f1) tuples"""
    avg_precision = np.mean([m[0] for m in metrics])
    avg_recall = np.mean([m[1] for m in metrics])
    avg_f1 = np.mean([m[2] for m in metrics])
    
    return {
        'precision': avg_precision,
        'recall': avg_recall,
        'f1': avg_f1
    }

In [7]:
# Process the full training dataset
print("Starting full dataset processing...")
results, avg_metrics = process_dataset_in_batches(ds['train'], model)

print("\nFinal Average Metrics:")
print(f"Precision: {avg_metrics['precision']:.3f}")
print(f"Recall: {avg_metrics['recall']:.3f}")
print(f"F1 Score: {avg_metrics['f1']:.3f}")

Starting full dataset processing...


Processing samples:   0%|          | 0/8000 [00:00<?, ?it/s]


Saved results to keyphrase_results_1000_20241209_121608.pkl

Saved results to keyphrase_results_2000_20241209_121712.pkl

Saved results to keyphrase_results_3000_20241209_121817.pkl

Saved results to keyphrase_results_4000_20241209_121923.pkl

Saved results to keyphrase_results_5000_20241209_122030.pkl

Saved results to keyphrase_results_6000_20241209_122136.pkl

Saved results to keyphrase_results_7000_20241209_122241.pkl

Saved results to keyphrase_results_8000_20241209_122350.pkl

Total time taken: 522.57 seconds

Final Average Metrics:
Precision: 0.425
Recall: 0.425
F1 Score: 0.425


In [8]:
# Process the test dataset
print("Starting test dataset processing...")
test_results, test_metrics = process_dataset_in_batches(ds['test'], model)

print("\nTest Set Metrics:")
print(f"Precision: {test_metrics['precision']:.3f}")
print(f"Recall: {test_metrics['recall']:.3f}")
print(f"F1 Score: {test_metrics['f1']:.3f}")

Starting test dataset processing...


Processing samples:   0%|          | 0/2717 [00:00<?, ?it/s]


Saved results to keyphrase_results_1000_20241209_122457.pkl

Saved results to keyphrase_results_2000_20241209_122603.pkl

Total time taken: 180.60 seconds

Test Set Metrics:
Precision: 0.423
Recall: 0.423
F1 Score: 0.423


In [9]:
# Load dataset 2
ds_inspec = load_dataset("midas/inspec", "generation")

# We'll only use train and test splits (ignoring validation)
train_samples = ds_inspec['train']
test_samples = ds_inspec['test']

# Take first 10 samples for testing (from train set)
test_samples = list(train_samples.select(range(10)))

# Display first sample structure
print("Sample data structure:")
print(test_samples[0])

# Display all 10 samples
print("\nFirst 10 samples with their keyphrases:")
for i, sample in enumerate(test_samples):
    print(f"\n{i+1}. Document preview: {' '.join(sample['document'][:20])}...")
    print(f"   Extractive keyphrases: {sample['extractive_keyphrases']}")

Sample data structure:
{'id': 1001, 'document': ['A', 'conflict', 'between', 'language', 'and', 'atomistic', 'information', 'Fred', 'Dretske', 'and', 'Jerry', 'Fodor', 'are', 'responsible', 'for', 'popularizing', 'three', 'well-known', 'theses', 'in', 'contemporary', 'philosophy', 'of', 'mind', ':', 'the', 'thesis', 'of', 'Information-Based', 'Semantics', '-LRB-', 'IBS', '-RRB-', ',', 'the', 'thesis', 'of', 'Content', 'Atomism', '-LRB-', 'Atomism', '-RRB-', 'and', 'the', 'thesis', 'of', 'the', 'Language', 'of', 'Thought', '-LRB-', 'LOT', '-RRB-', '.', 'LOT', 'concerns', 'the', 'semantically', 'relevant', 'structure', 'of', 'representations', 'involved', 'in', 'cognitive', 'states', 'such', 'as', 'beliefs', 'and', 'desires', '.', 'It', 'maintains', 'that', 'all', 'such', 'representations', 'must', 'have', 'syntactic', 'structures', 'mirroring', 'the', 'structure', 'of', 'their', 'contents', '.', 'IBS', 'is', 'a', 'thesis', 'about', 'the', 'nature', 'of', 'the', 'relations', 'that', 'con

In [10]:
def prepare_text(tokens: List[str]) -> str:
    """Convert token list to string, removing special tokens"""
    cleaned_tokens = [t for t in tokens if not (t.startswith('-') and t.endswith('-'))]
    return ' '.join(cleaned_tokens)

def extract_keyphrases(text: str, model: KeyBERT, top_n: int = 8) -> List[str]:
    """Extract keyphrases using KeyBERT"""
    try:
        keyphrases = model.extract_keywords(text, 
                                          keyphrase_ngram_range=(1, 3),
                                          stop_words='english',
                                          top_n=top_n)
    except AttributeError:
        # For newer scikit-learn versions
        from sentence_transformers import SentenceTransformer
        sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
        model.model = sentence_model
        keyphrases = model.extract_keywords(text, 
                                          keyphrase_ngram_range=(1, 3),
                                          stop_words='english',
                                          top_n=top_n)
    
    return [k[0] for k in keyphrases]

# Initialize model
print("Loading KeyBERT model...")
model = KeyBERT()

Loading KeyBERT model...


In [11]:
# Test the keyphrase extraction on first few samples
for i, sample in enumerate(test_samples[:5]):  # Test first 5 samples
    # Prepare text
    text = prepare_text(sample['document'])
    
    # Extract keyphrases
    extracted_phrases = extract_keyphrases(text, model)
    
    # Print results
    print(f"\nSample {i+1}:")
    print(f"Document preview: {' '.join(sample['document'][:30])}...")
    print(f"True keyphrases: {sample['extractive_keyphrases']}")
    print(f"Extracted keyphrases: {extracted_phrases}")
    print("-" * 80)


Sample 1:
Document preview: A conflict between language and atomistic information Fred Dretske and Jerry Fodor are responsible for popularizing three well-known theses in contemporary philosophy of mind : the thesis of Information-Based Semantics...
True keyphrases: ['philosophy of mind', 'content atomism', 'ibs', 'language of thought', 'lot', 'cognitive states', 'beliefs', 'desires']
Extracted keyphrases: ['language atomistic information', 'atomistic information', 'language atomistic', 'atomism thesis language', 'content atomism atomism', 'conflict language atomistic', 'semantics ibs thesis', 'atomistic information fred']
--------------------------------------------------------------------------------

Sample 2:
Document preview: Selective representing and world-making We discuss the thesis of selective representing-the idea that the contents of the mental representations had by organisms are highly constrained by the biological niches within...
True keyphrases: ['selective represent

In [12]:
# Cell 4: Define evaluation metrics
def evaluate_matches(true_keyphrases: List[str], extracted_phrases: List[str], partial_match: bool = True) -> Tuple[float, float, float]:
    """Calculate precision, recall, and F1 score"""
    if partial_match:
        # Count each match only once by tracking which true keyphrases have been matched
        matched_true = set()
        matched_extracted = set()
        
        for i, ext in enumerate(extracted_phrases):
            for j, gold in enumerate(true_keyphrases):
                if (ext.lower() in gold.lower() or gold.lower() in ext.lower()):
                    matched_extracted.add(i)
                    matched_true.add(j)
        
        matches = len(matched_true)  # Count unique matches
    else:
        matches = sum(1 for ext in extracted_phrases 
                     if any(ext.lower() == gold.lower() for gold in true_keyphrases))
    
    precision = matches / len(extracted_phrases) if extracted_phrases else 0
    recall = matches / len(true_keyphrases) if true_keyphrases else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Evaluate sample results
all_metrics = []
for i, sample in enumerate(test_samples):
    # Prepare text and extract keyphrases
    text = prepare_text(sample['document'])
    extracted_phrases = extract_keyphrases(text, model)
    
    # Calculate metrics
    precision, recall, f1 = evaluate_matches(sample['extractive_keyphrases'], extracted_phrases)
    all_metrics.append((precision, recall, f1))
    
    # Print results
    print(f"\nSample {i+1}:")
    print(f"True keyphrases: {sample['extractive_keyphrases']}")
    print(f"Extracted phrases: {extracted_phrases}")
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}")

# Calculate average metrics
avg_precision = np.mean([m[0] for m in all_metrics])
avg_recall = np.mean([m[1] for m in all_metrics])
avg_f1 = np.mean([m[2] for m in all_metrics])

print("\nAverage Metrics:")
print(f"Precision: {avg_precision:.2f}")
print(f"Recall: {avg_recall:.2f}")
print(f"F1 Score: {avg_f1:.2f}")


Sample 1:
True keyphrases: ['philosophy of mind', 'content atomism', 'ibs', 'language of thought', 'lot', 'cognitive states', 'beliefs', 'desires']
Extracted phrases: ['language atomistic information', 'atomistic information', 'language atomistic', 'atomism thesis language', 'content atomism atomism', 'conflict language atomistic', 'semantics ibs thesis', 'atomistic information fred']
Precision: 0.25, Recall: 0.25, F1: 0.25

Sample 2:
True keyphrases: ['selective representing', 'mental representations', 'organisms', 'realism', 'cognitive profiles']
Extracted phrases: ['mental representations organisms', 'selective representing realism', 'selective representing world', 'representations organisms', 'representations organisms highly', 'realist conception mind', 'selective representing idea', 'mental representations']
Precision: 0.50, Recall: 0.80, F1: 0.62

Sample 3:
True keyphrases: ['classicism', 'universality', 'classical component of mind', 'human cognition', 'universal generalizatio

In [13]:
def process_dataset_in_batches(dataset, model, batch_size=200, save_every=1000):
    """Process the dataset in batches with progress monitoring and periodic saving"""
    start_time = time.time()
    all_metrics = []
    all_results = []
    total_samples = len(dataset)
    
    # Create progress bar
    pbar = tqdm(total=total_samples, desc="Processing samples")
    
    for i in range(0, total_samples, batch_size):
        # Get batch
        batch = dataset.select(range(i, min(i + batch_size, total_samples)))
        batch_metrics = []
        batch_results = []
        
        # Process each sample in batch
        for sample in batch:
            # Prepare text and extract keyphrases
            text = prepare_text(sample['document'])
            extracted_phrases = extract_keyphrases(text, model)
            
            # Calculate metrics
            precision, recall, f1 = evaluate_matches(sample['extractive_keyphrases'], 
                                                   extracted_phrases)
            batch_metrics.append((precision, recall, f1))
            
            # Store detailed results
            batch_results.append({
                'document': sample['document'],
                'true_keyphrases': sample['extractive_keyphrases'],
                'extracted_phrases': extracted_phrases,
                'metrics': {'precision': precision, 'recall': recall, 'f1': f1}
            })
        
        # Update main lists
        all_metrics.extend(batch_metrics)
        all_results.extend(batch_results)
        
        # Update progress bar
        pbar.update(len(batch))
        
        # Save periodically
        if (i + batch_size) % save_every == 0:
            save_results(all_results, all_metrics, i + batch_size)
            
    pbar.close()
    
    # Calculate final averages
    avg_metrics = calculate_average_metrics(all_metrics)
    
    # Print time taken
    time_taken = time.time() - start_time
    print(f"\nTotal time taken: {time_taken:.2f} seconds")
    
    return all_results, avg_metrics

def save_results(results, metrics, samples_processed):
    """Save results to file"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"inspec_results_{samples_processed}_{timestamp}.pkl"
    
    with open(filename, 'wb') as f:
        pickle.dump({
            'results': results,
            'metrics': metrics,
            'samples_processed': samples_processed
        }, f)
    print(f"\nSaved results to {filename}")

def calculate_average_metrics(metrics):
    """Calculate average metrics"""
    avg_precision = np.mean([m[0] for m in metrics])
    avg_recall = np.mean([m[1] for m in metrics])
    avg_f1 = np.mean([m[2] for m in metrics])
    
    return {
        'precision': avg_precision,
        'recall': avg_recall,
        'f1': avg_f1
    }

In [14]:
# Cell 6: Process full training dataset
print("Starting full training dataset processing...")
train_results, train_metrics = process_dataset_in_batches(ds_inspec['train'], model)
print("\nTraining Set Final Metrics:")
print(f"Precision: {train_metrics['precision']:.3f}")
print(f"Recall: {train_metrics['recall']:.3f}")
print(f"F1 Score: {train_metrics['f1']:.3f}")

Starting full training dataset processing...


Processing samples:   0%|          | 0/1000 [00:00<?, ?it/s]


Saved results to inspec_results_1000_20241209_123504.pkl

Total time taken: 482.16 seconds

Training Set Final Metrics:
Precision: 0.216
Recall: 0.335
F1 Score: 0.241


In [15]:
# Cell 7: Process test dataset
print("Starting test dataset processing...")
test_results, test_metrics = process_dataset_in_batches(ds_inspec['test'], model)
print("\nTest Set Metrics:")
print(f"Precision: {test_metrics['precision']:.3f}")
print(f"Recall: {test_metrics['recall']:.3f}")
print(f"F1 Score: {test_metrics['f1']:.3f}")

Starting test dataset processing...


Processing samples:   0%|          | 0/500 [00:00<?, ?it/s]


Total time taken: 233.89 seconds

Test Set Metrics:
Precision: 0.216
Recall: 0.321
F1 Score: 0.238


In [17]:
from tabulate import tabulate

# Collecting results from our experiments
def create_comparison_table(sem_eval_train, sem_eval_test, inspec_train, inspec_test):
    data = [
        ['SemEval-2010 Task 8', 'Train', 
         sem_eval_train['precision'], 
         sem_eval_train['recall'], 
         sem_eval_train['f1']],
        ['SemEval-2010 Task 8', 'Test', 
         sem_eval_test['precision'], 
         sem_eval_test['recall'], 
         sem_eval_test['f1']],
        ['Inspec', 'Train', 
         inspec_train['precision'], 
         inspec_train['recall'], 
         inspec_train['f1']],
        ['Inspec', 'Test', 
         inspec_test['precision'], 
         inspec_test['recall'], 
         inspec_test['f1']]
    ]
    
    headers = ['Dataset', 'Split', 'Precision', 'Recall', 'F1 Score']
    print("\nBaseline Metrics:")
    print(tabulate(data, headers=headers, tablefmt='pipe', floatfmt='.3f'))

# Use our actual metrics from previous runs
create_comparison_table(
    sem_eval_train=avg_metrics,      # from SemEval train processing
    sem_eval_test=test_metrics,      # from SemEval test processing
    inspec_train=train_metrics,      # from Inspec train processing
    inspec_test=test_metrics        # from Inspec test processing
)


Baseline Metrics:
| Dataset             | Split   |   Precision |   Recall |   F1 Score |
|:--------------------|:--------|------------:|---------:|-----------:|
| SemEval-2010 Task 8 | Train   |       0.425 |    0.425 |      0.425 |
| SemEval-2010 Task 8 | Test    |       0.216 |    0.321 |      0.238 |
| Inspec              | Train   |       0.216 |    0.335 |      0.241 |
| Inspec              | Test    |       0.216 |    0.321 |      0.238 |
